<a href="https://colab.research.google.com/github/arthurziegler/ner-using-bert/blob/main/BERT_Experiment/bert_ner_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# First test of the BERT model for NER
Using [an implementation by Tobias Sterbak](https://www.depends-on-the-definition.com/named-entity-recognition-with-bert/) as a base for the tests of the application of bert in this problem.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange